The two-spiral problem is shown below.  
--------------------------

1. Randomly decompose the two-spiral problem into 4 sub-problems and use a Min-Max Modular neural network to solve the two-spiral problem. Each sub-module is a multilayer quadratic perceptron (MLQP) network with one hidden layer.


In [ ]:
# randomly decompose the problem into 4 sub-problem 
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

# load training data
train_path='data/two_spiral_train.txt'
train_data=np.loadtxt(train_path)
X=train_data[:,:2]
Y=train_data[:,-1]
print X.shape,Y.shape

# randomly decompose into 4 sub
# seperate 0,1 classes
X_0=X[Y==0,:]
X_1=X[Y==1,:]
print X_0.shape
# each seperate to two balance sub randomlu
index = np.random.permutation(X_0.shape[0])
X_0_0=X_0[index[:X_0.shape[0]/2],:]
X_0_1=X_0[index[X_0.shape[0]/2:],:]

index = np.random.permutation(X_1.shape[0])
X_1_0=X_1[index[:X_1.shape[0]/2],:]
X_1_1=X_1[index[X_1.shape[0]/2:],:]

Xd={0:X_0_0, 1:X_0_1,2:X_1_0, 3:X_1_1}
Yd={0:np.zeros((X_0_0.shape[0],1)),1:np.zeros((X_0_1.shape[0],1)),2:np.ones((X_1_0.shape[0],1)),3:np.ones((X_1_1.shape[0],1))}

# plot
plt.scatter(X[:,0],X[:,1],c=Y)
plt.axis('equal')
# 
plt.figure()
for i in range(4):
    plt.subplot(2,2,i+1)
    if i<2:
        plt.plot(Xd[i][:,0],Xd[i][:,1],'r.')
    else:
        plt.plot(Xd[i][:,0],Xd[i][:,1],'b.')
    plt.axis('equal')
#
plt.figure()
plt.subplot(141)
plt.plot(X_0_0[:,0],X_0_0[:,1],'r.')
plt.plot(X_1_0[:,0],X_1_0[:,1],'b.')
plt.axis('equal')
plt.subplot(142)
plt.plot(X_0_1[:,0],X_0_1[:,1],'r.')
plt.plot(X_1_1[:,0],X_1_1[:,1],'b.')
plt.axis('equal')
plt.subplot(143)
plt.plot(X_1_0[:,0],X_1_0[:,1],'b.')
plt.plot(X_0_1[:,0],X_0_1[:,1],'r.')
plt.axis('equal')
plt.subplot(144)
plt.plot(X_1_1[:,0],X_1_1[:,1],'b.')
plt.plot(X_0_0[:,0],X_0_0[:,1],'r.')
plt.axis('equal')



from multiprocessing import Pool

#  Min-Max Modular neural network 
def train(X,Y,flag):
    mlqps = MLQPs(n_input=X.shape[1],n_output=Y.shape[1],n_hidden=10) # the number of hidden unitss can be set to 10
    mlqps.train(X,Y,learning_rate=learning_rate,training_epoches=1e4, display_epoch=1000)
    print flag

for i in range(4):
    train(X[i],Y[i],i)
    
    
    

In [ ]:
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Pool(4)
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')

2. Plot the decision boundaries of each trained sub-module and the Min-Max Modular neural network.

3. Compare the training time and generalization performance of the above Min-Max Modular neural network and the single MLQP networks trained in Homework Assignment 1.

In [26]:
#------------ on-line learning for MLQPs -----------#
import numpy as np

# quadratic perceptron
class QP:
    def __init__(self,n_input,n_output):
        self.U=np.random.uniform(0,1,(n_input,n_output))
        self.V=np.random.uniform(0,1,(n_input,n_output))
        self.b=np.ones((1,n_output))
        self.one=np.ones(shape=self.b.shape)
        
    def forward(self,x):
        # x: the input data (1 x n_input)
        t=np.dot(x**2,self.U)+np.dot(x,self.V)+self.b
        self.y=1/(1+np.exp(-t))
        return self.y
    
    def backward(self,x,gy,lr):
        # gy: the gradient of y (1 x n_output) lr:learning_rate, for updating the parameterse
        gt=gy*self.y*(self.one-self.y) 

        gx=np.dot(gt,self.U.T)*2*x + np.dot(gt,self.V.T)
        gU=np.dot((x**2).T,gt)
        gV=np.dot(x.T,gt)
        gb=gt
        
        # update parameteres
        self.U=self.U-lr*gU
        self.V=self.V-lr*gV
        self.b=self.b-lr*gb
        
        return gx

# MLQPs
class MLQPs:
    def __init__(self,n_input,n_output,n_hidden=10):
        self.qp1=QP(n_input,n_hidden)
        self.qp2=QP(n_hidden,n_output)

    def forward(self,x):
        h=self.qp1.forward(x)
        pred=self.qp2.forward(h)
        return h,pred 
    
    def train(self,X,Y,learning_rate=0.05,training_epoches=1500,display_epoch=500):
        for epoch in np.arange(training_epoches):
            for i in range(X.shape[0]):
                x = X[i:(i+1),:] # on-line. training 1 instance each loop.
                y = Y[i:(i+1),:]
                # forward
                h,pred = self.forward(x)
                # L2 
                diff = pred-y
                cost = np.linalg.norm(diff)
                # backward
                # out layer
                gh = self.qp2.backward(h,diff,learning_rate)
                # hidden layer
                self.qp1.backward(x,gh,learning_rate)
            if cost<0.02:
                print 'Finished!  epoch: %d' % epoch,'cost: %f' % cost
                return 0
            if epoch % display_epoch == 0:
                print 'epoch: %d' % epoch,'cost: %f' % cost
        return 0

In [ ]:
plt.figure()
for i in range(4):
    plt.subplot(2,2,i+1)
    X_sub = X[index[i*bz:(i+1)*bz],:]
    Y_sub = Y[index[i*bz:(i+1)*bz]]
    plt.scatter(X_sub[:,0],X_sub[:,1],c=Y_sub)
plt.axis('equal')